# SETUP

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import psycopg2
from sqlalchemy import create_engine

# GET DATA

In [2]:
# fetch a web page
htlm_page = requests.get("https://www.francetvinfo.fr/sports/")
print(htlm_page.text[:1000])

<!DOCTYPE html><!--[if lt IE 7]><html class="no-js ie ie6 lte9 lte8 lte7" lang="fr"> <![endif]--><!--[if IE 7]><html class="no-js ie ie7 lte9 lte8 lte7" lang="fr"> <![endif]--><!--[if IE 8]><html class="no-js ie ie8 lte9 lte8" lang="fr"> <![endif]--><!--[if IE 9]><html class="no-js ie ie9 lte9" lang="fr"> <![endif]--><!--[if gt IE 9]><html class="no-js ie" lang="fr"> <![endif]--><!--[if !IE]><!--><html class="no-js" lang="fr"> <!--<![endif]--><head><title>Sports</title><meta http-equiv="pragma" content="no-cache" /><meta http-equiv="expires" content="Tue, 22 Feb 2000 12:14:57 GMT" /><meta http-equiv="refresh" content="1799" /><meta property="og:title" content="Sports" /><meta property="og:description" content="Suivez en direct toute l'actualité &quot;Sports&quot; : vivez l'info en live, en images et en vidéos. Participez, commentez et partager avec Franceinfo en temps réel !" /><meta property="og:type" content="website" /><meta property="og:locale:alternate" content="fr_FR" /><meta pro

In [3]:
page = BeautifulSoup(htlm_page.text, "html5lib")
print(page.get_text()[:1000])

Sports  var myHash='',queryString=document.location.search;var xtref=document.referrer.replace(/[<>]/g,'').replace(/&/g,'$');myHash='xtref='+((''!=xtref)?xtref:'acc_dir');if(document.location.search&&(new RegExp("xtor=([^&]+)")).test(document.location.search)){var xtor=(new RegExp("xtor=([^&]+)")).exec(document.location.search);if(xtor){myHash=((myHash!='')?'&':'')+'xtor='+xtor[1]} console.log('document.location.search',document.location.search);var queryString=document.location.search.replace(xtor[0],'');console.log("queryString",queryString);} myHash=(document.location.hash!='')?document.location.hash+((myHash!='')?'&'+myHash:''):'#'+myHash;(function(mobileUri){var isIpad=navigator.userAgent.match(/iPad/i)!=null,isMobile="undefined"!==typeof window.screen&&"undefined"!==typeof window.screen.width&&(768>=window.screen.width);if(isMobile&&!isIpad){window.location.href="//"+window.location.hostname.replace("www","mobile")+mobileUri}})(("/sports/"+queryString+myHash));  (function(){("und

## EXTRACT TITLES

In [4]:
# extract title
summaries = page.find_all('div', class_="flowItem")

In [5]:
summaries[0]

<div class="flowItem feature" entity="CMS-ExternalLink_3864713"><a href="https://sport.francetvinfo.fr/les-jeux-olympiques/ce-serait-lenfer-pour-nous-langoisse-des-jo-annules-pour-les-sports" rel="noopener" target="_blank"><figure id="image_3658577"><img alt="Le Lee Valley Centre, antre du canoë-kayak.&amp;nbsp;" botag="CMS-IMAGE_3658577" class="b-lazy" data-src="/image/75rw8vbgv-548c/840/472/21126117.jpg" height="472px" src="data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==" title="Le Lee Valley Centre, antre du canoë-kayak.  | FRANCEINFO" width="840px"/></figure><div class="right-content"><h2 class="title" field="title">"Ce serait l'enfer pour nous" : l'angoisse des JO annulés pour les sports confidentiels</h2><div class="description" field="description">L'éventuelle annulation des Jeux olympiques de Tokyo paraît encore improbable, mais la menace, même lointaine, se précise jour après jour. Certains athlètes, notamment dans les sports moins médiatisés, bâtiss

In [6]:
summaries[0].select_one("h2").get_text()

'"Ce serait l\'enfer pour nous" : l\'angoisse des JO annulés pour les sports confidentiels'

In [7]:
len(summaries)

28

In [8]:
sport_news = [x.select_one("h2").get_text() for x in summaries]

In [9]:
sport_news

['"Ce serait l\'enfer pour nous" : l\'angoisse des JO annulés pour les sports confidentiels',
 "Coronavirus : la Fédération française de football suspend l'ensemble des entraînements et compétitions",
 'Coronavirus : le sport mondial menacé',
 "Coronavirus : l'UEFA va proposer mardi une suspension de la Ligue des Champions et un report de l'Euro de football à 2021",
 "Retrouver le chemin de la plage pour les surfeurs après l'incident de Fukushima",
 'Football : le PSG remporte son match à huis clos à Dortmund',
 'Coronavirus : "Nous ne raisonnons ni sur l\'hypothèse d\'une annulation ni sur l\'hypothèse d\'un report" de Roland-Garros, affirme la FFT',
 "VIDEO. La mauvaise blague du basketteur Rudy Gobert, avant de découvrir qu'il est atteint du coronavirus",
 "Tokyo 2020 : neuf ans après le drame de Fukushima, le Japon n'a rien oublié",
 '"Les athlètes s’inquiètent" : l\'annulation des Jeux olympiques de Tokyo à cause du coronavirus en question',
 "DIRECT. JO 2020 : regardez la cérémon

# WRITE THE DATA IN POSTGRESQL DB

In [10]:
df = pd.DataFrame({"source": "franceinfo", "headline": sport_news, "date": pd.to_datetime('today').to_pydatetime().date()})

In [11]:
df.head()

,source,headline,date
0,franceinfo,"""Ce serait l'enfer pour nous"" : l'angoisse des...",2020-03-13
1,franceinfo,Coronavirus : la Fédération française de footb...,2020-03-13
2,franceinfo,Coronavirus : le sport mondial menacé,2020-03-13
3,franceinfo,Coronavirus : l'UEFA va proposer mardi une sus...,2020-03-13
4,franceinfo,Retrouver le chemin de la plage pour les surfe...,2020-03-13


In [12]:
# create an engine
alchemy_engine = create_engine('postgresql+psycopg2://antoinemertz@localhost/news', pool_recycle=3600)

In [13]:
conn = alchemy_engine.connect()
table = "sport"

In [22]:
already_in = pd.read_sql("SELECT * FROM  ;", conn)
[x for x in df["headline"] if x in already_in["headline"]]

In [14]:
try:
    already_in = pd.read_sql("SELECT headline FROM sport ;", conn)
    df.to_sql(name=table, con=conn, if_exists='append', index=False)
except Exception as ex:d
    print(ex)
conn.close()